### Pre-Configuring Database Connection

Update the following variables with your actual database credentials: username, password, host, port, and service name to establish a connection.


In [ ]:
import oracledb

username = 'your_username'
# IMPORTANT: do not put passwords into notebooks in plaintext.
# Instead, fetch it from a secure password store or if given by the user,
# use the password widget: https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html#password
password = 'your_password'
host = 'your_host'
port = 'your_port'
service_name = 'your_service_name'

dsn = f'{host}:{port}/{service_name}'
connection = oracledb.connect(user=username, password=password, dsn=dsn)
print("Connection established successfully.")

Please check this link for data, table creation https://github.com/oracle-samples/oracle-graph/tree/master/23c-graph-demos.

In [ ]:
# Set up the Graph

import opg4py

jdbc_url = f"jdbc:oracle:thin:@//{host}:{port}/{service_name}"

pgql_connection = opg4py.pgql.get_connection(username, password, jdbc_url)
pgql_statement = pgql_connection.create_statement()

create_pgql_graph = '''
    CREATE PROPERTY GRAPH BANK_GRAPH_PGQL
    VERTEX TABLES (
            BANK_ACCOUNTS
            KEY (ID)
            PROPERTIES (ID, Name, Balance) 
        )
        EDGE TABLES (
            BANK_TRANSFERS 
            KEY (TXN_ID) 
            SOURCE KEY (src_acct_id) REFERENCES BANK_ACCOUNTS(ID)
            DESTINATION KEY (dst_acct_id) REFERENCES BANK_ACCOUNTS(ID)
            PROPERTIES (src_acct_id, dst_acct_id, amount)
        )
    OPTIONS ( pg_view )
'''

    
pgql_result_set = pgql_statement.execute(create_pgql_graph)

pgql_statement.close()
pgql_connection.close()

### PgqlGraphVisualization

The following code block tests PgqlGraphVisualization features provided by the Jupyter extension.

**Note:** This page shows how to use a PGQL on RDBMS with VisualQuery to get visualization first and then assign it to GraphVisualization.data.


In [ ]:
from pypgx import setloglevel
from oraclegraph import GraphVisualization, PgqlGraphVisualization

setloglevel('ROOT', 'WARN')

vq = PgqlGraphVisualization(username, password, dsn)
query = 'SELECT * FROM MATCH (n) -[e]-> (m) ON BANK_GRAPH_PGQL'
graph_json = vq.visualize_query(query)
rule_based_styles = [
    {
        "component": "vertex",
        "target": "vertex",
        "conditions": {
            "conditions": [
                {
                    "property": "NAME",
                    "operator": "!=",
                    "value": ""
                }
            ]
        },
        "modifierStyles": {
            "hover": {
                "label": {
                    "text": "${properties.NAME}"
                }
            }
        },
        "legendTitle": "Hover - Name Caption"
    },
    {
        "component": "vertex",
        "target": "vertex",
        "conditions": {
            "conditions": [
                {
                    "property": "BALANCE",
                    "operator": ">=",
                    "value": 150000
                }
            ]
        },
        "style": {
            "color": "#AA5F71"
        },
        "legendTitle": "Balance >= $150,000"
    },
    {
        "component": "vertex",
        "target": "outgoing",
        "conditions": {
            "conditions": [
                {
                    "property": "BALANCE",
                    "operator": ">=",
                    "value": 150000
                }
            ]
        },
        "animations": [
            [
                {
                    "id": "animation_1",
                    "duration": 5,
                    "timingFunction": "linear",
                    "keyFrames": [
                        {
                            "percentage": 100,
                            "style": {
                                "strokeDashoffset": 100
                            }
                        }
                    ]
                }
            ]
        ],
        "style": {
            "color": "#825BBD",
            "dasharray": "dashed3"
        },
        "legendTitle": "Edge Animation"
    }
]

settings = {
    'ruleBasedStyles': rule_based_styles
}
graph = GraphVisualization(data=graph_json, settings=settings)
display(graph)
